In [112]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype    
 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib 

try:
    df = pd.read_csv('./cc_attach_project.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)
# df = df.sample(n=20000, random_state=1)

/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Success: Data loaded into dataframe.


In [118]:
df['COMMITMENTS'].value_counts()

0    1171512
1      32546
2       1524
3         30
Name: COMMITMENTS, dtype: int64

In [100]:

def combinations(cc_count, will_attach):
    if cc_count > 0 and will_attach == 1:
        return 0
    elif cc_count <= 0 and will_attach == 0:
        return 1
    elif cc_count > 0 and will_attach == 0:
        return 2
    else:
        return 3
    
to_drop = ['ORGANIZATION_ID', 'ENROLLMENT_DATE', 'DATE']
df.drop(to_drop, inplace=True, axis=1)

df['combination'] = df.apply(lambda row: combinations(row['CC_COUNT'], row['WILL_ATTACH_IN_60']), axis=1)
df['STAGENAME_PRODUCT'] = df['STAGENAME_PRODUCT'].fillna('not called')
df['INDUSTRY'] = df['INDUSTRY'].fillna('missing')
df['CASH_PERC'] = df['CASH_PERC'].fillna(0.0)  
df['CHECK_PERC'] = df['CHECK_PERC'].fillna(0.0)
df['OTHER_PERC'] = df['OTHER_PERC'].fillna(0.0)
df['ACH_PERC'] = df['ACH_PERC'].fillna(0.0)
df['CC_PERC'] = df['CC_PERC'].fillna(0.0)
df['BILLING_PLAN_ID'] = df['BILLING_PLAN_ID'].astype('int64')
cat_dtype = CategoricalDtype(categories=(df['PLAN_TIER'].unique()), ordered=True)
df['PLAN_TIER'] = df['PLAN_TIER'].astype(cat_dtype) #small > medium?
df['INDUSTRY'] = df['INDUSTRY'].astype('category')
df['ORGSIZE'] = df['ORGSIZE'].abs()
df['ORGSIZE'].replace(0,1, inplace=True) #should I do this?
#convert to all floats
df = df.apply(pd.to_numeric, errors='ignore', downcast='float')


In [101]:
for column in df:
    if df[column].dtype == np.float32 or df[column].dtype == np.int64:
        df[column] = df[column].astype('float64')

df['combination'] = pd.to_numeric(df['combination'], errors='raise', downcast='integer')

In [102]:
print(df.dtypes)

WILL_ATTACH_IN_60        float64
PLAN_TIER                 object
BILLING_PLAN_ID          float64
CC_FEE                   float64
ASP                      float64
INDUSTRY                  object
ORGSIZE                  float64
BANK_CONNECTED           float64
BANK_VERIFIED            float64
CREATED                  float64
SCHEDULED                float64
OMW                      float64
STARTED                  float64
IN_PROGRESS              float64
ESTIMATED                float64
INVOICED                 float64
TOTALPAYMENT_COUNT       float64
TOTALPAYMENT_AMOUNT      float64
ONLINE_BOOKINGS          float64
CC_COUNT                 float64
CC_AMOUNT                float64
INSTAPAY_AMOUNT          float64
ACH_COUNT                float64
CASH_COUNT               float64
CASH_AMOUNT              float64
CHECK_COUNT              float64
CHECK_AMOUNT             float64
OTHER_COUNT              float64
DAYS_SINCE_ENROLLMENT    float64
REASON_TO_ADOPT          float64
STAGENAME_

In [103]:
df.isna().sum()

WILL_ATTACH_IN_60        0
PLAN_TIER                0
BILLING_PLAN_ID          0
CC_FEE                   0
ASP                      0
INDUSTRY                 0
ORGSIZE                  0
BANK_CONNECTED           0
BANK_VERIFIED            0
CREATED                  0
SCHEDULED                0
OMW                      0
STARTED                  0
IN_PROGRESS              0
ESTIMATED                0
INVOICED                 0
TOTALPAYMENT_COUNT       0
TOTALPAYMENT_AMOUNT      0
ONLINE_BOOKINGS          0
CC_COUNT                 0
CC_AMOUNT                0
INSTAPAY_AMOUNT          0
ACH_COUNT                0
CASH_COUNT               0
CASH_AMOUNT              0
CHECK_COUNT              0
CHECK_AMOUNT             0
OTHER_COUNT              0
DAYS_SINCE_ENROLLMENT    0
REASON_TO_ADOPT          0
STAGENAME_PRODUCT        0
CC_COUNT_ALL             0
AVG_TIME_TO_PAYMENT      0
COMMITMENTS_ALL          0
COMMITMENTS              0
CC_PERC                  0
CASH_PERC                0
C

In [104]:
y = df['combination']
to_drop = ['combination']
df = df.drop(to_drop, axis=1)
X = pd.get_dummies(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
y_train= y_train.values.reshape(-1, 1)
y_test= y_test.values.reshape(-1, 1)

scaler_X = preprocessing.StandardScaler().fit(X_train)
scaler_y = preprocessing.StandardScaler().fit(y_train)
X_train_scaled = scaler_X.transform(X_train)
y_train_scaled = scaler_y.transform(y_train)



(16000, 154) (4000, 154) (16000,) (4000,)


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [107]:
clf = RandomForestRegressor(n_estimators=100, max_depth=None, max_features='sqrt',
                              random_state=0)

clf = make_pipeline(preprocessing.StandardScaler(), 
                         clf)


In [108]:
print(y_train.dtype, y_test.dtype)
clf.fit(X_train_scaled, y_train_scaled)


int8 int8


/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False))])

In [110]:
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test)
pred = clf.predict(X_test_scaled)

print(pred)

[-0.84974397 -0.87289184  0.77060718 ... -0.87289184 -0.87289184
  0.28450183]


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [111]:
r2_score(y_test_scaled, pred)


0.9571598920235654